In [1]:
import numpy as np
import pandas as pd
import urllib.request
import requests
import os
from sklearn.model_selection import train_test_split

In [2]:
df_georges = pd.read_csv('georges.csv')
df_non_georges = pd.read_csv('non_georges.csv')

In [3]:
df_georges.head()

,https://i.pinimg.com/736x/17/0d/5b/170d5b93d80d247be60f22ca1216bef7.jpg
0,https://i.pinimg.com/736x/47/b9/9a/47b99a2ddcd...
1,https://i.pinimg.com/736x/90/e8/90/90e890f054b...
2,https://i.pinimg.com/736x/0a/71/6f/0a716f6f14e...
3,https://i.pinimg.com/736x/f1/95/be/f195bea0b78...
4,https://i.pinimg.com/736x/78/da/54/78da54b8fa9...


In [4]:
df_non_georges.head()

,https://i.pinimg.com/736x/fd/65/bb/fd65bb679b7975ab73867d62fba2e991.jpg
0,https://i.pinimg.com/736x/78/63/a2/7863a2404af...
1,https://i.pinimg.com/736x/bc/3a/f4/bc3af41444c...
2,https://i.pinimg.com/736x/46/a3/59/46a3595eec7...
3,https://i.pinimg.com/736x/54/b2/ef/54b2ef7af71...
4,https://i.pinimg.com/736x/f6/f6/57/f6f6577d0cb...


In [5]:
print(df_georges.shape)
print(df_non_georges.shape)

(2680, 1)
(3365, 1)


In [6]:
df_georges.isnull().sum()

https://i.pinimg.com/736x/17/0d/5b/170d5b93d80d247be60f22ca1216bef7.jpg    0
dtype: int64

In [7]:
df_non_georges.isnull().sum()

https://i.pinimg.com/736x/fd/65/bb/fd65bb679b7975ab73867d62fba2e991.jpg    0
dtype: int64

In [10]:
root_path = os.getcwd()
folders = ['test', 'train', 'val']
sub_folders = ['georges', 'non_georges']
try:
    for folder in folders:
        for sub_name in sub_folders:
            os.makedirs(os.path.join(folder, sub_name))
except OSError:
    print ("Creation of the directory %s failed" % root_path)
else:
    print ("Successfully created the directory %s" % root_path)

Successfully created the directory C:\Users\kamikaze666\Python_3.7_projects\test_cv_engineer


In [15]:
g_x_train, g_x_test = train_test_split(df_georges, test_size=0.3)
g_x_test, g_x_val = train_test_split(g_x_test, test_size=0.5)

In [19]:
non_x_train, non_x_test = train_test_split(df_non_georges, test_size=0.3)
non_x_test, non_x_val = train_test_split(non_x_test, test_size=0.5)

In [20]:
print(g_x_train.shape)
print(g_x_test.shape)
print(g_x_val.shape)

(1876, 1)
(402, 1)
(402, 1)


In [21]:
print(non_x_train.shape)
print(non_x_test.shape)
print(non_x_val.shape)

(2355, 1)
(505, 1)
(505, 1)


In [23]:
print('{}\{}\{}'.format(root_path, folders[1],sub_folders[0]))

C:\Users\kamikaze666\Python_3.7_projects\test_cv_engineer\train\georges


# Download Georges train, test, val sets

In [28]:
def download_images(dataset, num_folder, num_subfolder):
    folders = ['test', 'train', 'val']
    sub_folders = ['georges', 'non_georges']
    
    for index, row in dataset.iterrows():
        r = requests.get(row[0])
        with open('{}\{}\{}'.format(folders[num_folder],
                                    sub_folders[num_subfolder],
                                    row[0].split('/')[-1]), 'wb') as f:
                                        f.write(r.content)

In [24]:
print('Beginning file download with requests')
for index, row in g_x_train.iterrows():
    r = requests.get(row[0])
    with open('{}\{}\{}'.format(folders[1],sub_folders[0], row[0].split('/')[-1]), 'wb') as f:
        f.write(r.content)

Beginning file download with requests


In [25]:
print('Beginning file download with requests')
for index, row in g_x_test.iterrows():
    r = requests.get(row[0])
    with open('{}\{}\{}'.format(folders[0],sub_folders[0], row[0].split('/')[-1]), 'wb') as f:
        f.write(r.content)

Beginning file download with requests


In [26]:
print('Beginning file download with requests')
for index, row in g_x_val.iterrows():
    r = requests.get(row[0])
    with open('{}\{}\{}'.format(folders[2],sub_folders[0], row[0].split('/')[-1]), 'wb') as f:
        f.write(r.content)

Beginning file download with requests


# Download non Georges train, test, val sets

In [31]:
download_images(non_x_train, 1, 1)

In [29]:
download_images(non_x_test, 0, 1)

In [30]:
download_images(non_x_val, 2, 1)

# Simple Net initialization

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [9]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [10]:
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__GPU: {}'.format(torch.cuda.is_available()))
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__CUDNN ENABLE: {}'.format(torch.backends.cudnn.enabled))
print('__Number CUDA Devices:', torch.cuda.device_count())

__Python VERSION: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
__pyTorch VERSION: 1.7.1
__GPU: True
__CUDNN VERSION: 7604
__CUDNN ENABLE: True
__Number CUDA Devices: 1


In [11]:
img_transforms = transforms.Compose([
                 transforms.Resize((64,64)),    
                 transforms.ToTensor(),
                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                       std=[0.229, 0.224, 0.225])])

In [12]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

batch_size=64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [13]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
simplenet = SimpleNet()

In [ ]:
simplenet.load_state_dict(torch.load("./tmp/simplenet.pth"))

In [15]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

In [16]:
check_image(val_data_path)

False

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

In [18]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [21]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 0.95, Validation Loss: 1.27, accuracy = 0.62
Epoch: 2, Training Loss: 0.84, Validation Loss: 0.62, accuracy = 0.65
Epoch: 3, Training Loss: 0.65, Validation Loss: 0.60, accuracy = 0.66
Epoch: 4, Training Loss: 0.63, Validation Loss: 0.59, accuracy = 0.66
Epoch: 5, Training Loss: 0.59, Validation Loss: 1.31, accuracy = 0.66
Epoch: 6, Training Loss: 0.73, Validation Loss: 2.01, accuracy = 0.65
Epoch: 7, Training Loss: 0.84, Validation Loss: 0.81, accuracy = 0.65
Epoch: 8, Training Loss: 0.67, Validation Loss: 1.00, accuracy = 0.67
Epoch: 9, Training Loss: 0.68, Validation Loss: 0.71, accuracy = 0.66
Epoch: 10, Training Loss: 0.66, Validation Loss: 0.59, accuracy = 0.67


In [24]:
torch.save(simplenet.state_dict(), os.path.join("./tmp", "simplenet.pth"))

In [22]:
labels = ['georges','non_georges']

img = Image.open("./test/georges/9c611ae2bd3af5d6ea686aef9252ff62.jpg") 
img = img_transforms(img).to(device)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction])

georges
